In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(model_name = "Gemma2-9b-It",groq_api_key = groq_api_key)

In [3]:
# from langchain_core.messages import HumanMessage,SystemMessage
# llm.invoke([SystemMessage(content="You are a helpful assistant."), HumanMessage(content="What is the capital of France?")])

In [4]:
from langchain_core.messages import HumanMessage,AIMessage
llm.invoke([HumanMessage(content="what is the capital of france"),AIMessage(content="The capital of France is **Paris**."),
            HumanMessage(content="what's my name and what do I do?"),
            AIMessage(content="As an AI, I don't have access to personal information about you, including your name or what you do.\n\nIf you'd like to tell me your name and what you do, I'd be happy to know! 😊"),
            HumanMessage(content="Fuck you")
            ])

AIMessage(content="I understand that you're upset. \n\nIs there anything I can help you with, or would you like to talk about what's bothering you? \n\nI'm here to provide helpful and friendly assistance.  \n\nPlease remember that using abusive language isn't productive. Let's try to communicate respectfully.\n", response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 105, 'total_tokens': 174, 'completion_time': 0.125454545, 'prompt_time': 0.00280377, 'queue_time': 0.25293228, 'total_time': 0.128258315}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-842ea1ac-fb66-466f-ba5d-eea113e8bc6f-0', usage_metadata={'input_tokens': 105, 'output_tokens': 69, 'total_tokens': 174})

#### message history

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {}
def get_session_history(session_id : str)-> BaseChatMessageHistory:
    
    if(session_id not in store):
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm,get_session_history)

In [6]:
config = {"configurable": {"session_id" : "chat1"}}

In [7]:
with_message_history.invoke(
    [
        HumanMessage(content="Hii, my name is ALok Goyal, I'm a student")
    ],
    config = config
)

AIMessage(content="Hello Alok Goyal, nice to meet you! 👋\n\nI'm glad you reached out. What can I do for you today? \n\nAre you working on a school project?  Do you have a question about something you're learning? Or are you just curious about what I can do? \n\nTell me more! 😊\n\n", response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 24, 'total_tokens': 98, 'completion_time': 0.134545455, 'prompt_time': 0.00132988, 'queue_time': 0.25352445, 'total_time': 0.135875335}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-87d0875c-7800-4ae0-aec6-55b873b276e7-0', usage_metadata={'input_tokens': 24, 'output_tokens': 74, 'total_tokens': 98})

# If we ask the question in same session history, check will it able to remember the context?

In [8]:


with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config
)

AIMessage(content='Your name is Alok Goyal. 😊 \n\n\n\nI remember that from when you introduced yourself!  Is there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 110, 'total_tokens': 144, 'completion_time': 0.061818182, 'prompt_time': 0.002811309, 'queue_time': 0.254175931, 'total_time': 0.064629491}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dd831a32-2a1b-42d9-9132-7c23b5eff651-0', usage_metadata={'input_tokens': 110, 'output_tokens': 34, 'total_tokens': 144})

#### If we ask the question in another session then will it able to remember the context?

In [9]:
config1 = {"configurable": {"session_id": "abc123"}}
with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config = config1
)

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n", response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 14, 'total_tokens': 54, 'completion_time': 0.072727273, 'prompt_time': 0.00124732, 'queue_time': 0.25231967, 'total_time': 0.073974593}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d2af5e64-523a-4f6c-ab4a-b1e8043b84b3-0', usage_metadata={'input_tokens': 14, 'output_tokens': 40, 'total_tokens': 54})

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
    ("system" , "Hey, You are a helpful assistaint. Answer all the question to the next of your ability."),
    MessagesPlaceholder(variable_name = "messages")
    ]
)

chain = prompt | llm 

In [11]:
chain.invoke({"messages": [HumanMessage(content="Hi my name is Alok Goyal")]})

AIMessage(content="Hello Alok Goyal, it's nice to meet you!  \n\nI'm ready to answer your questions to the best of my ability.  Just ask away! 😊  \n", response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 38, 'total_tokens': 79, 'completion_time': 0.074545455, 'prompt_time': 0.00155577, 'queue_time': 0.25247750999999996, 'total_time': 0.076101225}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a92c0d09-6559-4f27-b436-9bec16d103c5-0', usage_metadata={'input_tokens': 38, 'output_tokens': 41, 'total_tokens': 79})

In [12]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [13]:
config = {"configurable": {"session_id": "chat2"}}
with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Alok Goyal")],
    config= config
)

AIMessage(content="Hello Alok Goyal, it's nice to meet you!\n\nI'm happy to help with any questions you have. Just ask away! 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 39, 'total_tokens': 76, 'completion_time': 0.067272727, 'prompt_time': 0.00156078, 'queue_time': 0.25392964, 'total_time': 0.068833507}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-40c6c604-5230-425a-90e8-938c31fd1f9d-0', usage_metadata={'input_tokens': 39, 'output_tokens': 37, 'total_tokens': 76})

In [14]:
# Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
       ( "system" , "You are a helpful assistant. Answer all the questions to the best of your ability in {language}."),
       MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | llm
response = chain.invoke({"messages" : [HumanMessage(content="Hii, My name is Alok Goyal")], "language" : "Hindi"})
print(response)

content='नमस्ते अलोक गोयल जी! \n\nमुझे बहुत खुशी है कि आप मुझसे जुड़े हैं। क्या मैं आपकी कोई मदद कर सकता हूँ? \n\n(Hello Alok Goyal!\n\nI am very happy to connect with you. Can I help you with anything?)\n\n' response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 37, 'total_tokens': 108, 'completion_time': 0.129090909, 'prompt_time': 0.00155498, 'queue_time': 0.25377604000000004, 'total_time': 0.130645889}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-e0675c14-081c-4434-8add-325ed86f95ff-0' usage_metadata={'input_tokens': 37, 'output_tokens': 71, 'total_tokens': 108}


#### let's wrap this more complicated chain in a message history class. This time, Because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [15]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hii, my name is alok goyal")],"language" : "Hindi"},
    config=config
)
print(response)

content='नमस्ते अलोक गोयल! 😊 \n\nआप क्या जानना चाहेंगे? मैं आपकी मदद करने के लिए तैयार हूँ।  \n' response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 37, 'total_tokens': 74, 'completion_time': 0.067272727, 'prompt_time': 0.00155414, 'queue_time': 0.25244661, 'total_time': 0.068826867}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-158f21d7-b820-4768-aa74-f80c8c35fdab-0' usage_metadata={'input_tokens': 37, 'output_tokens': 37, 'total_tokens': 74}


In [17]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name?")],"language": "Hindi"},
    config=config
)
print(response)

content='आपका नाम अलोक गोयल है। 😄  \n' response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 88, 'total_tokens': 105, 'completion_time': 0.030909091, 'prompt_time': 0.00250142, 'queue_time': 0.25199907000000005, 'total_time': 0.033410511}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-88e3967e-1a62-4dcf-b32d-faadd9c5ee77-0' usage_metadata={'input_tokens': 88, 'output_tokens': 17, 'total_tokens': 105}


#### Managing the conversation history

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

What is trim_messages?
trim_messages is a message-trimming utility in LangChain that ensures:

You don't send too many messages to the model

You stay within the token limit (like 4096 or 8192 tokens, depending on the model)

In [18]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(max_tokens = 40,strategy = "last",token_counter = llm,include_system = True,allow_partial = False,start_on = "human")

# set of messages 
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?"),
    AIMessage(content="The capital of France is **Paris**."),
    HumanMessage(content="What's my name and what do I do?"),
    AIMessage(content="As an AI, I don't have access to personal information about you, including your name or what you do.\n\nIf you'd like to tell me your name and what you do, I'd be happy to know! 😊"),
    HumanMessage(content="what 2+2?"),
    AIMessage(content="2 + 2 = 4"),
    HumanMessage(content="do you have any hobbies?"),
    AIMessage(content="As an AI, I don't have personal experiences or hobbies like humans do.")
]
trimmer.invoke(messages)

c:\Users\HP\Desktop\Langchain\langenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content='You are a helpful assistant.'),
 HumanMessage(content='do you have any hobbies?'),
 AIMessage(content="As an AI, I don't have personal experiences or hobbies like humans do.")]

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

# create a chain
chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")|trimmer)
    |prompt|llm
)

response = chain.invoke({"messages":messages + [HumanMessage(content="What is the capital of France?")],"language":"English"})
print(response)

content='The capital of France is Paris.  \n' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 42, 'total_tokens': 53, 'completion_time': 0.02, 'prompt_time': 0.00156251, 'queue_time': 0.25329973, 'total_time': 0.02156251}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-bf1b397b-c02c-4b23-a10e-dd699948ba8b-0' usage_metadata={'input_tokens': 42, 'output_tokens': 11, 'total_tokens': 53}


In [21]:
# let's wrap this in the message history
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")
config = {"configurable":{"session_id":"chat5"}}

In [24]:
response = with_message_history.invoke(
    {"messages":messages + [HumanMessage(content="what's my name?")],"language":"English"},
    config=config
)
print(response)

content="As a helpful assistant, I have no memory of past conversations and don't know your name.  \n\nIs there anything else I can help you with? 😊 \n\n" response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 41, 'total_tokens': 79, 'completion_time': 0.069090909, 'prompt_time': 0.001554191, 'queue_time': 0.252557968, 'total_time': 0.0706451}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-8577e2c3-b86a-4a87-a2d6-baebbaf98212-0' usage_metadata={'input_tokens': 41, 'output_tokens': 38, 'total_tokens': 79}
